In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
from sklearn.metrics import classification_report

In [2]:
iris = load_iris()

In [3]:
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=['y'])

In [4]:
df = pd.DataFrame(X)
df['y'] = y

In [5]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),y
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
class Perceptron:
    def __init__(self, df):
        x, y = df.loc[:, df.columns!='y'], df['y']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)
        self.x_train = x_train
        self.x_train.insert(0,'x0',1)
        self.x_test = x_test
        self.x_test.insert(0,'x0',1)
        self.y_train = y_train
        self.y_test = y_test
        self.d_features = len(x.columns)
        self.n_datapoints = self.x_train.shape[0]
        self.l_classes = len(set(y))
        self.Ws = [np.array([0 for _ in range(self.d_features+1)]) for _ in range(self.l_classes)]

    def display_attr(self):
        # print(self.Ws)
        for i, elt in enumerate(self.Ws):
            print(i, elt)

    def softmax(self, index, output_neuron_values):
        numerator = np.exp(output_neuron_values[index])
        denominator = 0

        for val in output_neuron_values:
            denominator += np.exp(val)

        return numerator/denominator

    def argmax(self, output_neuron_values):
        maxidx = -1
        maxval = -1e12

        for i, elt in enumerate(output_neuron_values):
            if elt>maxval:
                maxval = elt
                maxidx = i

        return maxidx

    def build_model(self):
        n = self.n_datapoints
        d = self.d_features
        l = self.l_classes

        Wolds = self.Ws[:]
        Wnews = self.Ws[:]
        epochs = 0
        eta = random.choice(np.linspace(0, 0.1, 1000))

        x = self.x_train
        y = self.y_train

        printer_steps = set(int(np.floor(i)) for i in np.linspace(0, n, 50))

        while True:
            if epochs!=0:
                for i, elt in enumerate(Wolds):
                    Wnews[i] = elt

            if epochs%1000==0:
                print(f"Running Epoch {epochs+1} ", end='')

            for i in range(n):
                if epochs%1000==0 and i in printer_steps:
                    print("==", end='')

                x_vec = x.iloc[i]
                y_class = y.iloc[i]
                y_true = 0

                output_neuron_values = []

                for j in range(l):
                    Wj = Wolds[j]
                    WjTx = Wj.T.dot(x_vec)
                    output_neuron_values.append(WjTx)

                for j in range(l):
                    y_true = 1 if j==y_class else 0

                    softmax_probability = self.softmax(j, output_neuron_values)
                    Wolds[j] = Wolds[j] - eta*(softmax_probability-y_true)*np.array(x_vec)

            if epochs%1000==0:
                print(">")
                for i, elt in enumerate(Wolds):
                    print(f"Old vs New Weight of Hyperplane{i+1}")
                    print(f"Wold {i} {Wolds[i]}, Wnew {i} {Wnews[i]}")

            brkflg = True
            epochs += 1

            # print(Wolds, Wnews)
            for i, elt in enumerate(Wolds):
                if not np.allclose(Wolds[i],Wnews[i]):
                    brkflg = False

            if brkflg:
                break

        self.Ws = Wnews

        print(f"Model training Successful at epochs {epochs}")
        print(f"Final Weights: ")
        for i, wi in enumerate(self.Ws):
            print(i, wi)

    def test_model(self):
        x = self.x_test
        n = x.shape[0]
        Ws = self.Ws
        l = self.l_classes
        y_trues = np.array(self.y_test)
        y_preds = []

        for i in range(n):
            x_vec = x.iloc[i]
            y_true = 0

            output_neuron_values = []

            for j in range(l):
                Wj = Ws[j]
                WjTx = Wj.T.dot(x_vec)
                output_neuron_values.append(WjTx)

            y_preds.append(self.argmax(output_neuron_values))

        y_preds = np.array(y_preds)

        print(classification_report(y_trues, y_preds))

In [7]:
x, y = df.loc[:, df.columns!='y'], df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33)

In [8]:
x_train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
149,5.9,3.0,5.1,1.8
95,5.7,3.0,4.2,1.2
73,6.1,2.8,4.7,1.2
58,6.6,2.9,4.6,1.3
129,7.2,3.0,5.8,1.6
...,...,...,...,...
100,6.3,3.3,6.0,2.5
145,6.7,3.0,5.2,2.3
114,5.8,2.8,5.1,2.4
59,5.2,2.7,3.9,1.4


In [9]:
p1 = Perceptron(df)

In [10]:
p1.display_attr()

0 [0 0 0 0 0]
1 [0 0 0 0 0]
2 [0 0 0 0 0]


In [11]:
np.exp(1)

2.718281828459045

In [12]:
p1.build_model()

Running Epoch 1 ==================================================================================================>
Old vs New Weight of Hyperplane1
Wold 0 [ 0.01177575  0.00113825  0.06356688 -0.14177754 -0.0605749 ], Wnew 0 [0 0 0 0 0]
Old vs New Weight of Hyperplane2
Wold 1 [-0.00472445 -0.02197193 -0.03678857  0.01921574  0.00193987], Wnew 1 [0 0 0 0 0]
Old vs New Weight of Hyperplane3
Wold 2 [-0.0070513   0.02083368 -0.02677831  0.1225618   0.05863503], Wnew 2 [0 0 0 0 0]
Running Epoch 1001 ==================================================================================================>
Old vs New Weight of Hyperplane1
Wold 0 [ 0.61393475  1.35509521  2.69728763 -3.86011901 -1.7704876 ], Wnew 0 [ 0.61376343  1.35468999  2.69659689 -3.859113   -1.76999585]
Old vs New Weight of Hyperplane2
Wold 1 [ 1.38551918  0.93743898  0.02106072 -0.45242604 -1.54058224], Wnew 1 [ 1.38457037  0.93727149  0.0209471  -0.45225209 -1.54006096]
Old vs New Weight of Hyperplane3
Wold 2 [-1.99945393 -2

In [13]:
p1.test_model()

In [14]:
p1.Ws

[array([ 1.58852285,  3.58074783,  5.65155573, -8.81250728, -4.38796444]),
 array([15.6701828 , -0.52387814,  0.06526203,  0.29453219, -5.10884564]),
 array([-17.25870564,  -3.0568697 ,  -5.71681776,   8.5179751 ,
          9.49681008])]

In [37]:
x = p1.x_test
y = p1.y_test
n = x.shape[0]
Ws = p1.Ws
l = p1.l_classes
y_trues = np.array(p1.y_test)
y_preds = []
y_pred_probs = []

for i in range(n):
    x_vec = x.iloc[i]
    y_true = 0
    y_class = y.iloc[i]

    output_neuron_values = []

    for j in range(l):
        Wj = Ws[j]
        WjTx = Wj.T.dot(x_vec)
        output_neuron_values.append(WjTx)

    y_preds.append(p1.argmax(output_neuron_values))

    softmax_probabilities = []

    for j in range(l):
        y_true = 1 if j==y_class else 0
        softmax_probability = p1.softmax(j, output_neuron_values)
        softmax_probabilities.append(softmax_probability)

    y_pred_probs.append(softmax_probabilities)

y_preds = np.array(y_preds)
y_pred_probs = np.array(y_pred_probs)

print(classification_report(y_trues, y_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        15

    accuracy                           1.00        50
   macro avg       1.00      1.00      1.00        50
weighted avg       1.00      1.00      1.00        50



In [40]:
from sklearn.metrics import roc_auc_score
# 1v1
roc_auc_score(y_trues, y_pred_probs, multi_class='ovo')

1.0

In [41]:
roc_auc_score(y_trues, y_pred_probs, multi_class='ovr')

1.0